In [ ]:
# from pydantic_settings import BaseSettings, SettingsConfigDict

# class Settings(BaseSettings):
#     GOOGLE_API_KEY: str
#     model_config = SettingsConfigDict(env_file=".env")

# env = Settings()

In [ ]:
# from langchain_google_genai import GoogleGenerativeAIEmbeddings

# embeddings_2 = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001", google_api_key=env.GOOGLE_API_KEY)

In [ ]:
%pip install pydantic_settings langchain langchain-core langchain-google-genai langchain-qdrant fastembed langchain-community qdrant-client langgraph

## SETUP

In [34]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings

embeddings_2 = FastEmbedEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    cache_dir="./embedding_cache"
)


In [35]:
from qdrant_client.http.models import Distance

collection_name = "doctor_packages"
dimension = 384  
distance = Distance.COSINE

In [36]:
import json

# Load doctor data
with open('doctors_final.json', 'r', encoding='utf-8') as f:
    doctors_data = json.load(f)

print(doctors_data[0])

{'id': '5eae0017-40dd-4961-869f-79d9e45d87f2', 'name': 'Adventia Emilia Krysna Sipi Seda, M.M., M.Psi., Psikolog', 'specialization_name': 'Psikologi', 'specialization_name_en': 'Psychology', 'sub_specialization_name': 'Psikolog', 'sub_specialization_name_en': 'Psychologist', 'hospital_name': 'Siloam Hospitals Yogyakarta'}


In [37]:
from qdrant_client import QdrantClient

client = QdrantClient(":memory:")

In [38]:
from qdrant_client.http.models import VectorParams

if client.collection_exists(collection_name=collection_name):
    client.delete_collection(collection_name=collection_name)


client.create_collection(
    collection_name=collection_name,
    vectors_config=VectorParams(size=dimension, distance=distance),
)

True

In [39]:
# Fix the vector data creation cell
from qdrant_client.models import PointStruct
import uuid

for index, row in enumerate(doctors_data):
    
    if not row.get('name') or not row.get('specialization_name'):
        continue
        
    text = f"Doctor Name: {row['name']}, Description: Spesialisasi: {row['specialization_name']} ({row.get('specialization_name_en', '')}), Sub-spesialisasi: {row.get('sub_specialization_name', '')}, Rumah Sakit: {row.get('hospital_name', '')}"
    
    # Generate embedding for the text
    emb = embeddings_2.embed_query(text)
    
    client.upsert(
        collection_name=collection_name,
        points=[
            PointStruct(
                id=str(uuid.uuid4()),
                vector=emb,
                payload={
                    "page_content": text,
                    "metadata": {
                        "id": row.get('id',''),
                        "name": row.get('name',''),
                        "specialization_name": row.get('specialization_name',''),
                        "specialization_name_en": row.get('specialization_name_en', ''),
                        "sub_specialization_name": row.get('sub_specialization_name', ''),
                        "hospital_name": row.get('hospital_name', ''),
                    },
                },
            )
        ],
    )
    print(f"{index + 1}. {text}")

1. Doctor Name: Adventia Emilia Krysna Sipi Seda, M.M., M.Psi., Psikolog, Description: Spesialisasi: Psikologi (Psychology), Sub-spesialisasi: Psikolog, Rumah Sakit: Siloam Hospitals Yogyakarta
2. Doctor Name: Dokter Umum Siloam Yogyakarta, Description: Spesialisasi: Kedokteran Umum (General Practitioner), Sub-spesialisasi: Dokter Umum, Rumah Sakit: Siloam Hospitals Yogyakarta
3. Doctor Name: dr. Andrean Jefrian Manihuruk, Description: Spesialisasi: Kedokteran Umum (General Practitioner), Sub-spesialisasi: Dokter Umum, Rumah Sakit: Siloam Hospitals Yogyakarta
4. Doctor Name: dr. Anggoro Budi Hartopo, M.Sc., Ph.D., SpPD., SpJP (K), Description: Spesialisasi: Kardiologi (Jantung) (Cardiology (Heart)), Sub-spesialisasi: Subspesialis Prevensi dan Rehabilitasi Kardiovaskular, Rumah Sakit: Siloam Hospitals Yogyakarta
5. Doctor Name: dr. Antonius Arif Kusuma, SpB, Description: Spesialisasi: Bedah Umum (General Surgery), Sub-spesialisasi: Spesialis Bedah, Rumah Sakit: Siloam Hospitals Yogyakar

In [47]:
# Create retriever function
from langchain_qdrant import QdrantVectorStore

def get_retriever():
    vector_store = QdrantVectorStore(
        client=client,
        collection_name=collection_name,
        embedding=embeddings_2,
    )
    
    return vector_store.as_retriever()

In [44]:
# Create search tool for doctors
from langchain_core.tools import tool
from typing import Annotated, List

@tool
def search_doctors(query: Annotated[str, "search query must contain keywords related to doctor specialization, hospital, or medical condition"]) -> List[str]:
    """Search for doctors by specialization, hospital, or medical condition."""
    retriever = get_retriever()
    results = retriever.invoke(query, k=5)
    return [result.page_content for result in results]

In [41]:
# Initialize LLM
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    api_key=env.GOOGLE_API_KEY,
)

# Test the basic search functionality
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that provides information about Siloam hospitals and doctors."),
    ("human", "{question}"),
])

chain = prompt | llm

In [42]:
# Test basic LLM without RAG
test_response = chain.invoke({"question": "Saya butuh dokter jantung di Siloam Yogyakarta"})
print(test_response.content)

Tentu, saya dapat membantu. Berikut adalah dokter jantung yang berpraktik di Siloam Yogyakarta:

*   **dr. Aris Wibudi, Sp.JP**

Untuk membuat janji temu atau mendapatkan informasi lebih lanjut, Anda dapat menghubungi Siloam Hospitals Yogyakarta secara langsung.


In [48]:
# Test search tool
search_results = search_doctors("dokter jantung siloam yogyakarta cardiology")
for i, result in enumerate(search_results, 1):
    print(f"{i}. {result}")
    print("-" * 80)

1. Doctor Name: dr. Surya Prabowo, SpJP, M.Kes, FIHA, Description: Spesialisasi: Kardiologi (Jantung) (Cardiology (Heart)), Sub-spesialisasi: Spesialis Jantung dan Pembuluh Darah, Rumah Sakit: Siloam Hospitals Yogyakarta
--------------------------------------------------------------------------------
2. Doctor Name: dr. Royhan Rozqie, Ph.D, SpJP, Description: Spesialisasi: Kardiologi (Jantung) (Cardiology (Heart)), Sub-spesialisasi: Spesialis Jantung dan Pembuluh Darah, Rumah Sakit: Siloam Hospitals Yogyakarta
--------------------------------------------------------------------------------
3. Doctor Name: dr. Firman, SpJP, Description: Spesialisasi: Kardiologi (Jantung) (Cardiology (Heart)), Sub-spesialisasi: Spesialis Jantung dan Pembuluh Darah, Rumah Sakit: Siloam Hospitals Makassar
--------------------------------------------------------------------------------
4. Doctor Name: dr. Emanoel Oepangat, SpJP, K, Description: Spesialisasi: Kardiologi (Jantung) (Cardiology (Heart)), Sub-sp

In [49]:
# Create workflow states
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

class State(TypedDict):
    question: str
    context: List[str]
    search: str
    answer: str

In [50]:
def retrieve(state: State):
    retrieved_docs = search_doctors(state["search"])
    return {"context": retrieved_docs}

In [51]:
# Define workflow functions
def get_context(state: State):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
            Anda adalah ahli dalam pencarian dokter di rumah sakit Siloam.
            Anda akan memberikan kata kunci tentang pencarian dokter berdasarkan pertanyaan.
            Kata kunci harus relevan dengan spesialisasi dokter, nama rumah sakit, atau kondisi medis.
            Jangan berikan informasi lain.
            Jika pertanyaan sudah mengandung kata kunci, Anda dapat mengembalikannya apa adanya.
            Berikan kata kunci dalam bahasa Indonesia dan Inggris jika memungkinkan.
        """),
        ("human", "{question}"),
    ])
    
    chain = prompt | llm
    result = chain.invoke({"question": state["question"]})
    return {"search": result.content}

In [52]:
def generate(state: State):
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
            Anda adalah asisten yang membantu mencari dokter yang tepat di rumah sakit Siloam.
            
            Berdasarkan informasi dokter yang diberikan dalam konteks, berikan rekomendasi dokter yang paling sesuai dengan kebutuhan pasien.
            
            Format jawaban Anda dengan:
            1. Nama dokter lengkap
            2. Spesialisasi dan sub-spesialisasi
            3. Rumah sakit tempat praktik
            4. Penjelasan singkat mengapa dokter tersebut sesuai
            
            Jika tidak ada dokter yang sesuai dalam konteks, berikan saran umum atau minta pasien untuk mencari di spesialisasi yang lebih umum.
            
            Berikan jawaban dalam bahasa Indonesia yang ramah dan informatif.
        """),
        ("human", "Pertanyaan: {question}\n\nInformasi Dokter:\n{context}"),
    ])
    
    context_str = "\n\n".join(state["context"])
    chain = prompt | llm
    result = chain.invoke({
        "question": state["question"], 
        "context": context_str
    })
    return {"answer": result.content}

In [53]:
# Build the workflow graph
graph_builder = StateGraph(State).add_sequence([get_context, retrieve, generate])
graph_builder.add_edge(START, "get_context")
graph = graph_builder.compile()

In [56]:
# Test the complete workflow
test_questions = [
    "Saya butuh dokter kulit di Siloam Yogyakarta",
    "Ada dokter anak di Siloam?",
    "Cari dokter jantung yang bagus",
    "Dokter spesialis mata di Siloam Medan"
]

for question in test_questions:
    print(f"Pertanyaan: {question}")
    print("=" * 50)
    
    response = graph.invoke({
        "question": question,
        "context": [],
        "search": "",
        "answer": ""
    })
    
    print(f"Jawaban: {response['answer']}")
    print("-" * 80)
    print()

Pertanyaan: Saya butuh dokter kulit di Siloam Yogyakarta
Jawaban: Tentu, saya akan bantu memberikan rekomendasi dokter kulit di Siloam Yogyakarta:

1.  **dr. Dina Putri Chandra, SpKK**
    *   Spesialisasi: Dermatologi (Kulit), Sub-spesialisasi: Spesialis Dermatologi Venereologi Estetika
    *   Rumah Sakit: Siloam Hospitals Yogyakarta
    *   **Penjelasan:** dr. Dina adalah seorang spesialis kulit yang berpraktik di Siloam Yogyakarta dengan sub-spesialisasi di bidang estetik.

2.  **dr. Anna Kautsaria Putri, M.Med.Sc, SpDVE**
    *   Spesialisasi: Dermatologi (Kulit), Sub-spesialisasi: Spesialis Dermatologi Venereologi Estetika
    *   Rumah Sakit: Siloam Hospitals Yogyakarta
    *   **Penjelasan:** dr. Anna juga merupakan spesialis kulit di Siloam Yogyakarta dengan fokus pada bidang estetik.

3.  **dr. Monika Puspitasari, M.Med.Sc, SpDVE**
    *   Spesialisasi: Dermatologi (Kulit), Sub-spesialisasi: Spesialis Dermatologi Venereologi Estetika
    *   Rumah Sakit: Siloam Hospitals Yogy

In [55]:
# Create interactive function for easy testing
def find_doctor(question: str):
    """
    Function to find doctors based on user question
    """
    response = graph.invoke({
        "question": question,
        "context": [],
        "search": "",
        "answer": ""
    })
    
    return response["answer"]

# Test the function
print("=== SILOAM DOCTOR FINDER ===")
print()

# Example usage
user_question = "Saya membutuhkan dokter spesialis jantung untuk konsultasi tentang tekanan darah tinggi"
result = find_doctor(user_question)

print(f"Pertanyaan: {user_question}")
print(f"Rekomendasi: {result}")

=== SILOAM DOCTOR FINDER ===

Pertanyaan: Saya membutuhkan dokter spesialis jantung untuk konsultasi tentang tekanan darah tinggi
Rekomendasi: Tentu, dengan senang hati saya akan memberikan rekomendasi dokter spesialis jantung yang sesuai dengan kebutuhan Anda:

1.  **dr. Aldila Pradityawati, SpJP**
2.  Spesialisasi: Kardiologi (Jantung), Sub-spesialisasi: Spesialis Jantung dan Pembuluh Darah
3.  Rumah Sakit: Siloam Hospitals Bekasi Timur
4.  **Penjelasan:** dr. Aldila Pradityawati adalah seorang spesialis jantung dan pembuluh darah yang praktik di Siloam Hospitals Bekasi Timur. Dengan keahliannya di bidang kardiologi, beliau dapat membantu Anda dalam konsultasi dan penanganan masalah tekanan darah tinggi.

Semoga informasi ini membantu Anda! Jika Anda memiliki pertanyaan lebih lanjut atau membutuhkan informasi tambahan, jangan ragu untuk bertanya.
